In [1]:
# EDA
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.float_format', lambda x: '%.2f' % x) # Dica: para visualizar os valores continuos muito altos sem formato cientifico

# ML
from sklearn.cluster import KMeans # Pacocte de Clusterização
from sklearn.metrics import silhouette_score, pairwise_distances # ALgoritmos de Clusterização
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder # Normalizar dados numericos de escalas diferentes, variaveis categorias e variaveis ordinais
from sklearn.compose import ColumnTransformer # Transformar algumas colunas antes da clusterização

import optuna

### Carregar dados

In [2]:
df_clientes = pd.read_csv('.\dataset\clientes_pj.csv')

<>:1: SyntaxWarning:

invalid escape sequence '\d'

<>:1: SyntaxWarning:

invalid escape sequence '\d'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_19032\3996757017.py:1: SyntaxWarning:

invalid escape sequence '\d'



In [3]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [5]:
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


### EDA

In [7]:
# Distribuição da variavel inovação em percentual
percentual_inovacao = df_clientes.value_counts('inovacao') / len(df_clientes) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

In [ ]:
# Teste ANOVA (Análise de Variância)
# Verificar se há variações significativas na média de faturamento mensal para diferentes níveis de inovação
# Suposições / Pressupostos:
# - Observações independentes
# - Variável dependente é continua
# - Segue uma distribuição normal
# - Homogenidade das Variâncias
# - Amostras sejam de tamanhos iguais


In [8]:
# Checar se as variâncias (faturamento) entre os grupos (inovação) são homogênias
# Aplicar Teste de Bartlett
# H0 - Variâncias são iguais
# H1 - Variâncias não são iguais

from scipy.stats import bartlett

# Separando os dados de faturamento em grupos com base na coluna 'inovação'
dados_agrupados = [df_clientes['faturamento_mensal'][df_clientes['inovacao'] == grupo] for grupo in df_clientes['inovacao'].unique()] # Cria-se listas de faturamento com base em cada nivel de inovação

# Executar teste de Bartlett
bartlett_test_statistc, bartlett_p_value = bartlett(*dados_agrupados) # Retorna dois parametros depois realizar a função que recebera a lista quebrada em varias listas como argumento da função

# Exibindo resultados
print(f'Estatistica do Test de Bartlett: {bartlett_test_statistc}')
print(f'P-value do Test de Bartlett: {bartlett_p_value}')

Estatistica do Test de Bartlett: 10.901203117231173
P-value do Test de Bartlett: 0.28254182954905804


- temos que aceitar H0, pois o p-value é maior que 0.05

In [9]:
# Executar o Teste de Shapiro-Wilk
# Verificar se os dados seguem uma distribuição normal
# H0 - Segue uma distribuição normal
# H1 - Não segue uma distribuição normal

from scipy.stats import shapiro

# Executar o teste
shapiro_test_statistc, shapiro_p_value = shapiro(df_clientes['faturamento_mensal'])

# Exibindo Resultados
print(f'Estatistica do Teste SW: {shapiro_test_statistc}')
print(f'P-Value do Teste SW: {shapiro_p_value}')

Estatistica do Teste SW: 0.9959857602472711
P-Value do Teste SW: 0.23513451034389005


- P-Value maior que 0.05 confirma H0

In [10]:
# Aplicar a ANOVA de Welch, pois as amostras são de tamanhos diferentes
# H0 - Não há diferenças significativas entres as médias dos grupos
# H1 - Há pelo menos um diferenças significativas entres as médias dos grupos
from pingouin import welch_anova

aov = welch_anova(dv='faturamento_mensal', between='inovacao', data=df_clientes)

# Exibindo Resultados
print(f'Estatistica do Teste de ANOVA de Welch: {aov.loc[0, 'F']}')
print(f'P-Value do Teste de ANOVA de Welch: {aov.loc[0, 'p-unc']}')

Estatistica do Teste de ANOVA de Welch: 1.1269836194061693
P-Value do Teste de ANOVA de Welch: 0.34526211273911467


- P-value > 0.05 temos que aceitar H0